In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, load_model
import numpy as np
import pandas as pd
import shutil
import cv2 as cv2
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import os

In [ ]:
def predictor(sdir, csv_path,  model_path, crop_image = False):    
    # read in the csv file
    class_df=pd.read_csv(csv_path)    
    img_height=int(class_df['height'].iloc[0])
    img_width =int(class_df['width'].iloc[0])
    img_size=(img_width, img_height)
    scale=class_df['scale by'].iloc[0] 
    try: 
        s=int(scale)
        s2=1
        s1=0
    except:
        split=scale.split('-')
        s1=float(split[1])
        s2=float(split[0].split('*')[1]) 
        print (s1,s2)
    path_list=[]
    paths=os.listdir(sdir)
    for f in paths:
        path_list.append(os.path.join(sdir,f))
    print (' Model is being loaded- this will take about 10 seconds')
    model=load_model(model_path)
    image_count=len(path_list)    
    index_list=[] 
    prob_list=[]
    cropped_image_list=[]
    good_image_count=0
    for i in range (image_count):       
        img=cv2.imread(path_list[i])        
        good_image_count +=1
        img=cv2.resize(img, img_size)            
        cropped_image_list.append(img)
        img=img*s2 - s1
        img=np.expand_dims(img, axis=0)
        p= np.squeeze (model.predict(img))           
        index=np.argmax(p)            
        prob=p[index]
        index_list.append(index)
        prob_list.append(prob)
    if good_image_count==1:
        class_name= class_df['class'].iloc[index_list[0]]
        probability= prob_list[0]
        img=cropped_image_list [0] 
        plt.title(class_name, color='blue', fontsize=16)
        plt.axis('off')
        rgb_img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(rgb_img)
        return class_name, probability
    elif good_image_count == 0:
        return None, None
    most=0
    for i in range (len(index_list)-1):
        key= index_list[i]
        keycount=0
        for j in range (i+1, len(index_list)):
            nkey= index_list[j]            
            if nkey == key:
                keycount +=1                
        if keycount> most:
            most=keycount
            isave=i             
    best_index=index_list[isave]    
    psum=0
    bestsum=0
    for i in range (len(index_list)):
        psum += prob_list[i]
        if index_list[i]==best_index:
            bestsum += prob_list[i]  
    img= cropped_image_list[isave] 
    rgb_img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    class_name=class_df['class'].iloc[best_index]
    plt.title(class_name, color='blue', fontsize=16)
    plt.axis('off')
    plt.imshow(rgb_img)
    return class_name, bestsum/image_count

### Predict a single image of AFRICAN CROWNED CRANE

In [ ]:
csv_path=r'../input/100-bird-species/285 birds/class_dict.csv' # path to class_dict.csv
model_path=r'../input/100-bird-species/285 birds/EfficientNetB3-birds-98.87.h5' # path to the trained model
sdir=r'../input/100-bird-species/285 birds/one image to test'
class_name, probability=predictor(sdir, csv_path,  model_path, crop_image = False) # run the classifier
msg=f' image is an {class_name} with a probability of {probability * 100: 6.2f} %'
print(msg)

### input 5 images and get the species prediction and averaged probability
### Note in real use the directory should contain multiple images of the SAME bird
### however I did not have multiple images  of the same bird
### so I used 5 AFRICAN CROWNED CRANE images

In [ ]:
sdir=r'../input/100-bird-species/285 birds/images to test'
class_name, probability=predictor(sdir, csv_path,  model_path, crop_image = False) # run the classifier
msg=f' images are predicted as  {class_name} with a averaged probability of {probability * 100: 6.2f} %'
print(msg)